# **1. About the Dataset**

##### ERA5

# **2. Get Weather Data**

In [4]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import ee

In [9]:
# GEE Authentification
ee.Authenticate()

True

In [10]:
#GEE initialization
ee.Initialize()

In [39]:
def extract_weather(coords,sd,ed):

    era5 = ee.ImageCollection("ECMWF/ERA5/DAILY")
    # Coords
    site = ee.Geometry.Polygon(coords).centroid()
    weather_collection =  era5.filterBounds(site).filterDate(sd,ed).select(['minimum_2m_air_temperature','maximum_2m_air_temperature','total_precipitation'])
    # From Image collection to Image
    weather_results = []
    for i in range(weather_collection.size().getInfo()):
        weather_image = ee.Image(weather_collection.toList(weather_collection.size()).get(i))
        #Reduce the Weather Image
        mean_weather_image = weather_image.reduceRegion(
            reducer = ee.Reducer.mean(),
            geometry = site,
            scale = 27830
        )
        #Transfrom the image to Dataframe
        mean_weather_df = mean_weather_image.getInfo()
        # Get datetime from the image
        date_str = weather_image.date().format('YYYY-MM-dd').getInfo()
        date_time_weather_image = pd.to_datetime(date_str)
        
        # Store the data into the results list
        weather_results.append({
            'datetime': date_time_weather_image ,
            'Tmin(C)':   mean_weather_df.get('minimum_2m_air_temperature'),
            'Tmax(C)':   mean_weather_df.get('maximum_2m_air_temperature')   ,
            'Prcp(mm)':  mean_weather_df.get('total_precipitation') 
        })
        

    weather_data = pd.DataFrame(weather_results)
    weather_data['Tmin(C)'] = weather_data['Tmin(C)'] -273.15  
    weather_data['Tmax(C)'] = weather_data['Tmax(C)']-273.15 
    weather_data['Prcp(mm)'] = weather_data['Prcp(mm)']*1000

    return weather_data
        
        

    

    

# **3. Example** 

In [41]:
coordinates =[[[-7.608118848227721,31.658398246914924],
               [-7.608977155112484,31.65901924150963],
               [-7.611187295340757,31.656644828010087],
               [-7.610328988455992,31.656115142875652],
               [-7.608118848227721,31.658398246914924]]]
start_date = '2010-01-01'
end_date = '2010-12-31'
weather = extract_weather(coordinates, start_date, end_date)
weather


,datetime,Tmin(C),Tmax(C),Prcp(mm)
0,2010-01-01,8.232416,15.556512,0.271402
1,2010-01-02,5.897089,19.656122,0.000000
2,2010-01-03,7.718256,21.024469,0.002313
3,2010-01-04,8.233942,20.103357,1.062382
4,2010-01-05,11.257776,16.051233,1.452392
...,...,...,...,...
359,2010-12-26,9.447076,18.244348,0.122514
360,2010-12-27,6.939020,21.185663,0.000000
361,2010-12-28,9.129236,23.032922,0.000000
362,2010-12-29,8.742548,24.557153,0.000000


In [47]:
 weather[['Tmin(C)','Tmax(C)','Prcp(mm)']].describe()

,Tmin(C),Tmax(C),Prcp(mm)
count,364.000000,364.000000,364.000000
mean,14.315308,26.856316,0.939025
std,5.341294,7.349318,2.896639
min,2.431573,10.936578,0.000000
25%,10.091982,21.215189,0.000000
50%,13.448846,26.286676,0.006545
75%,18.585466,32.132295,0.567473
max,27.340936,44.517969,38.290560
